In [1]:
import pytorch_lightning as L

/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from data_loading import load_data, get_loaders, get_probe_data, get_probe_loaders, make_dataset
from transformer_predictor import TransformerPredictor, train, test

In [3]:
dataset = load_data('brackets.csv')

In [4]:
train_loader, val_loader, test_loader = get_loaders(dataset, batch_size=64)

In [7]:
model = TransformerPredictor(
    input_dim=4,
    model_dim=128,
    num_classes=2,
    num_heads=4,
    num_layers=2,
    lr=1e-3,
    warmup=100,
    max_iters=1000,
)
trainer = L.Trainer(max_epochs=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
model, trainer, train_outbeddings, val_outbeddings = train(model, trainer, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                | Type               | Params
-----------------------------------------------------------
0 | input_net           | Sequential         | 640   
1 | positional_encoding | PositionalEncoding | 0     
2 | transformer         | TransformerEncoder | 264 K 
3 | output_net          | Sequential         | 17.0 K
4 | accuracy            | BinaryAccuracy     | 0     
-----------------------------------------------------------
282 K     Trainable params
0         Non-trainable params
282 K     Total params
1.131     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [9]:
test_outbeddings = test(model, trainer, test_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9854271411895752     │
│         test_loss         │    0.03745344653725624    │
└───────────────────────────┴───────────────────────────┘

In [10]:
X_probe_train, y_probe_train = get_probe_data(train_outbeddings)
X_probe_val, y_probe_val = get_probe_data(val_outbeddings)
X_probe_test, y_probe_test = get_probe_data(test_outbeddings)

train_dataset = make_dataset(X_probe_train, y_probe_train)
val_dataset = make_dataset(X_probe_val, y_probe_val)
test_dataset = make_dataset(X_probe_test, y_probe_test)

train_probe_loader, val_probe_loader, test_probe_loader = get_probe_loaders(train_dataset, val_dataset, test_dataset, batch_size=64)

In [30]:
from probe import Probe

In [31]:
probe = Probe(model_name='lr')
# probe.fit(train_probe_loader, val_probe_loader)
probe.probe(X_probe_train.cpu().detach().numpy(), y_probe_train.cpu().detach().numpy(), X_probe_test.cpu().detach().numpy(), y_probe_test.cpu().detach().numpy())

              precision    recall  f1-score   support

           5       1.00      0.67      0.80        33
          15       0.90      1.00      0.95       104

    accuracy                           0.92       137
   macro avg       0.95      0.83      0.87       137
weighted avg       0.93      0.92      0.91       137



/home2/adyansh/dinner_pool/rsai/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [32]:
probe.plot_decision_boundary(X_probe_test.cpu().detach().numpy(), y_probe_test.cpu().detach().numpy())

ValueError: X has 2 features, but LogisticRegression is expecting 128 features as input.